<a href="https://colab.research.google.com/github/Doryane/Web-Scrapping-Archive-of-our-own-AO3-/blob/main/Web_Scrapping_AO3_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
start_time = time.time()
#main()

Web Scripping pour AO3 based on following code :
https://github.com/kenalba/ao3-scraper/blob/main/scraper.py



---

# What this code do ?

With this code you can append the following information from AO3 and put it into a data frame from a certain page to another

*  Story IDS of the work
*  Updated date of the work
*  Title and author of the work
*  Language of the work
*  Number of words in the work
*  Number of chapter in the work
*  Number of hits 
*  Requiered tags 
*  Additionnal tags

This code do the same as Web Srapping AO3 part 1 but only to append the database.

It was supposed to be quicker than part 1 but it's not and I don't really get why.

EDIT : So actually this code is indeed faster when you append more than 50 page, I tried to append 100 page and this code (part 2) took 368 seconds and the other (part 1) 395 seconds to run (without the graph and histogram etc).



---
# ⚠ WARNING ⚠
Two important thing to understand when you download **many files** of data :


1.   **You can't append more than 70 pages in one time**

  If you want to append a lot of data I suggest 50 page by 50 page. I mean if you want to try to append more that's your choice but the code is optimized to append 50 by 50 (or less).

  Also, one page is 20 work, so each time you run the code you can append at max 1400 works.

2.   **You need to have the SAME interval of page between files**

  Per example, you want to append the work from a ertain fandom, you have 457 pages in total. I would suggest to append page 1 to 50 (base_page_0_to_51), then 50 to 100 (base_page_50_to_101), ... , then 400 to 450 (base_page_400_to_450 and finally page 450 to 456 (base_page_450_to_457).

  For the last one you input 456 as Number_page but its append untill page 457 because my code is WEIRD.




This section doesn't have a lot of comments, if you're curious on how the code work I suggest you to check part 1 (it's the same result at the end)

**Library**

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import csv
from time import sleep
import pickle
from os import path
import pandas as pd
import datetime
import random
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from google.colab import files
import pandas as pd

**Parameters**

Just as part 1, we ask you to input an URL, a first page and a last one.

In [3]:
# Choose the fandom, the beginning of the page and the ending of the page of the data you want to append

# ULR
TEST_URL = "https://archiveofourown.org/tags/Stranger%20Things%20(TV%202016)/"

# First_Page
First_Page = 1
Beginning_Page = First_Page

# Last_page
Last_page = 100
Last_page = Last_page + 1
Number_page = Last_page

In [4]:
# Cleaning the tag
tag= urlparse(TEST_URL)
tag = tag.path
tag_2 = tag
tag_2 = tag_2.replace('tags','')
tag_2 = tag_2.replace('/','')
tag_2 = tag_2.replace('%20',' ')
tag_2 = tag_2.replace('*s*',' and ')

In [5]:
print("Download Data for the following tag : ",tag_2,"from page",First_Page,"to",Last_page)

Download Data for the following tag :  Stranger Things (TV 2016) from page 1 to 101


**Define functions**

Download and soupify

In [6]:
def download_and_soupify(url, parser="html.parser"):
	"""
	Given a URL, downloads the site and turns it into beautiful soup.
	"""

	full_url = url + "works"
	response = urllib.request.urlopen(full_url)
	directory_html = response.read()
	index_soup = BeautifulSoup(directory_html, parser)

	return index_soup

Get directory urls

In [7]:
def get_directory_urls(url, index_soup):
	"""
	Given the base directory as soup, figure out how many pages there are,
	and return a list of the URLs of each directory page.
	"""

	page_numbers = index_soup.find_all("ol", class_="pagination actions")

	li_entries = page_numbers[0].find_all("li")

	li_texts = [number.text for number in li_entries]
	li_digits = [int(number) for number in li_texts if number.isdigit()]
	sorted_page_numbers = sorted(li_digits)

	number_of_pages = sorted_page_numbers[-1]

	url_prefix = url + "works?page="
	directory_urls = [(url_prefix + str(page_number)) for page_number in range(0, number_of_pages+1)]

	return directory_urls

Story IDS

In [8]:
def get_story_ids(directory_url):
	"""
	Given a single directory URL, get all stories from that URL. Returns a list of story IDs.
	"""

	dir_soup = download_and_soupify(directory_url)
	dir_links = dir_soup.find_all("a", href=True)

	dir_hrefs = [link.attrs['href'] for link in dir_links]
	work_hrefs = [link for link in dir_hrefs if "/works/" in link]
	potential_ids = [work.split("/")[2] for work in work_hrefs]
	id_list = [int(work_id) for work_id in potential_ids if work_id.isdigit()]

	story_ids = list(set(id_list))

	return story_ids

Date update of the work

In [9]:
def datetime(directory_url):
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("p",{"class":"datetime"}):
    text=x.get_text()
    date_work.append(str(text))

Title and author of the work

In [10]:
def worktitle(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("h4",attrs={"class": "heading"}):  
    text=x.get_text()
    work_title_authors.append(str(text))

  work_title_authors.remove('\n            Include\n          ')
  work_title_authors.remove('\n            Exclude\n          ')
  work_title_authors.remove('More Options')
  work_title_authors.remove('Pages Navigation')
  work_title_authors.remove('About the Archive')
  work_title_authors.remove('Pages Navigation')
  work_title_authors.remove('Contact Us')
  work_title_authors.remove('Development')

  (work_title_authors)

# It quit the same as the other function 
# Exept this time we need to remove a lot of stuff by page to clean the text


Language of the work

In [11]:
def worklanguage(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "language"}):  
    text=x.get_text()
    work_language.append(str(text))

  (work_language)

# I guess you know how it works now

Number of words of the work

In [12]:
def NumberofWords(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "words"}):  
    text=x.get_text()
    Nbr_words.append(str(text))

Number of Chapter

In [13]:
def NumberofChapter(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "chapters"}):  
    text=x.get_text()
    Nbr_chapter.append(str(text))

Number of Hits

In [14]:
def NumberofHits(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "hits"}):  
    text=x.get_text()
    Nbr_hits.append(str(text))

Requiered tags

In [15]:
def tags_req(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("ul",attrs={"class": "required-tags"}):  
    text=x.get_text()
    tags_requiered.append(str(text))

Additionals tags

In [16]:
def tags(directory_url):
  
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("ul",attrs={"class": "tags commas"}):  
    text=x.get_text()
    tags_add.append(str(text))

**Big function for everything**

In [17]:
index_soup = download_and_soupify(TEST_URL)

In [18]:
directory_urls = get_directory_urls(TEST_URL,index_soup)

In [19]:
# Empty storage
story_ids = []
story_ids_2 = []

date_work = []

work_title_authors = []

work_language = []
work_language_2 = []

Nbr_words = []

Nbr_chapter = []

Nbr_hits = []

tags_requiered = []

tags_add = []


index_page = []

In [20]:
def data_append(URL, Beginning_Page, Number_page):
  
  for i in range(Beginning_Page,Number_page):
    # story_ids_2
    story_ids.append(get_story_ids(directory_urls[i]))

    # data_updated
    date_work.append(datetime(directory_urls[i]))
    date_work.remove(None)

    # work_title_authors
    work_title_authors.append(worktitle(directory_urls[i]))
    work_title_authors.remove(None)

    # work_language
    work_language.append(worklanguage(directory_urls[i]))
    work_language.remove(None)

    # Number of words
    Nbr_words.append(NumberofWords(directory_urls[i]))
    Nbr_words.remove(None)

    # Number of chapter
    Nbr_chapter.append(NumberofChapter(directory_urls[i]))
    Nbr_chapter.remove(None)

    # Number of hits
    Nbr_hits.append(NumberofHits(directory_urls[i]))
    Nbr_hits.remove(None)

    # Requiered tags
    tags_requiered.append(tags_req(directory_urls[i]))
    tags_requiered.remove(None)

    # Additionnal tags
    tags_add.append(tags(directory_urls[i]))
    tags_add.remove(None)

###############################################################################

    # index_page
    for j in range(20):
      index_page.append(i)

  # work_title_authors
  for work in range(len(work_title_authors)):
    work_title_authors[work] = work_title_authors[work].replace('\n',' ')
    work_title_authors[work] = work_title_authors[work].replace('       ',' ')
    work_title_authors[work] = work_title_authors[work].replace('     ',' ')

  # work_language
  Number_page_bis = Number_page - Beginning_Page
  Number_page_bis = Number_page_bis
  Drop_language_setting = []
  for i in range(Number_page_bis):
    Drop_language_setting_bis = 20
    Drop_language_setting_bis = Drop_language_setting_bis + i*20
    Drop_language_setting.append(Drop_language_setting_bis)
  Drop_language_setting

  for i in Drop_language_setting:
    work_language.remove(work_language[i])
  
  # Requiered tags and additionnal tags
  for i in range(len(tags_requiered)):
    tags_requiered[i] = tags_requiered[i].replace("\n","|")
    tags_add[i] = tags_add[i].replace("\n","|")
    backslash = '"\\"'
    tags_add[i] = tags_add[i].replace(backslash,'')

**Running the function**

In [21]:
data_append(TEST_URL,First_Page,Last_page)

In [22]:
for i in range(0,len(story_ids)):
 story_ids_2 = story_ids_2 + story_ids[i]

# Adjust on story_ids 

**Inside a DataFrame**

In [23]:
# Dataframe
base1 = {'Index_page' : index_page, 
          'ID' :story_ids_2,
          'Date_publication' : date_work, 
          'Title_and_author' : work_title_authors, 
          'Language_of_the_work' : work_language, 
          'Word' : Nbr_words,
          'Chapter' : Nbr_chapter
          , 'Hits' : Nbr_hits
          , 'Requiered_tags' : tags_requiered
          , 'Additionnal_tags' : tags_add}
base1 = pd.DataFrame(data=base1)
base1

,Index_page,ID,Date_publication,Title_and_author,Language_of_the_work,Word,Chapter,Hits,Requiered_tags,Additionnal_tags
0,1,44157319,02 Feb 2023,Harringrove Stuff by wrecked_fuse,English,0,60/60,111004,| Explicit| Choose Not To Use Archive Warnings...,|Creator Chose Not To Use Archive Warnings No ...
1,1,44733832,02 Feb 2023,"The Past, The Future and The Present by Sodap...",English,"2,278",1/2,0,"| Explicit| Underage| M/M, Multi| Work in Prog...",|UnderageWill Byers/Mike Wheeler Mike Wheeler/...
2,1,21311245,02 Feb 2023,Like Magnets by Pondermoniums for wrecked_fuse,English,"25,522",18/?,7588,| Teen And Up Audiences| No Archive Warnings A...,|No Archive Warnings ApplyBilly Hargrove/Steve...
3,1,44511886,02 Feb 2023,Be Patient by TATAMONIQUELAGINETTE,English,"2,395",1/1,0,| Explicit| No Archive Warnings Apply| F/F| Co...,|No Archive Warnings ApplyRobin Buckley/Nancy ...
4,1,43212331,02 Feb 2023,Midnight (Belongs to You and Me) by Polkadotd...,English,"24,448",4/?,1254,| Explicit| Choose Not To Use Archive Warnings...,|Creator Chose Not To Use Archive WarningsChri...
...,...,...,...,...,...,...,...,...,...,...
1995,100,44318299,22 Jan 2023,something happens and i'm head over heels by ...,English,"12,694",5/?,1063,| Explicit| Choose Not To Use Archive Warnings...,|Creator Chose Not To Use Archive WarningsStev...
1996,100,40882788,22 Jan 2023,Love of a Lifetime by dibidibidani,English,"19,053",7/?,667,| Teen And Up Audiences| No Archive Warnings A...,|No Archive Warnings ApplyChrissy Cunningham/E...
1997,100,44463469,22 Jan 2023,Home by the Sea by WreakingHavok,English,"11,716",4/5,1965,| General Audiences| Choose Not To Use Archive...,|Creator Chose Not To Use Archive WarningsWill...
1998,100,44462842,22 Jan 2023,Asking You to Love Me by chattrekisses,English,"22,397",1/1,1999,| Explicit| No Archive Warnings Apply| M/M| Co...,|No Archive Warnings ApplySteve Harrington/Edd...


In [24]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 368.374635219574 seconds ---


**Export the DataBase**

Download your database on your computer

In [25]:
base_export = base1
Name_base = "base_page_" + str(Beginning_Page) + "_to_" + str(Number_page) + ".csv"
Name_base

'base_page_1_to_101.csv'

In [26]:
"""
from google.colab import files
base_export.to_csv(Name_base) 
files.download(Name_base)
"""

'\nfrom google.colab import files\nbase_export.to_csv(Name_base) \nfiles.download(Name_base)\n'